<a href="https://colab.research.google.com/github/FedericoRaschiatore0123/GATr_Deep_Learning_Project/blob/main/GA_sample_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import h5py
import os
import pandas as pd
import seaborn as sns

In [10]:
# create a dictionary of global variables
global_variables = {

    'dim_GA': 16
}

# Geometric Algebra Embedding

Geometric algebra G_{3,0,1} is characterized by 4 basic elements (e_{0}, e_{1}, e_{2}, e_{3}) that represent planes in the 3D space. These 4 basic elements can be combined through linear combination and geometric transformation to represent geometric objects of different dimensions (planes, lines, points), and transformations of different types (reflections, translations, rotations).

G_{3,0,1} indicates that the basis is composed by three elements (e_{1}, e_{2}, e_{3}) that satisfy e_{i}e_{i} = 1 and one (e_{0}) that satisfies e_{0}e_{0} = 0.

The basis components of the multivector associated to the G_{3,0,1} algebra are obtained by considering every possible multiplicative combination of the basis elements. Elements of different grade are thus obtained:

Grade 0 (1): scalars in the G.A. \\
Grade 1 (e_{0}, e_{i}): vectors in the G.A. \\
Grade 2 (e_{0i}, e_{ij}); bivectors in the G.A. \\
Grade 3 (e_{0ij}, e_{ijk}): trivectors in the G.A. \\
Grade 4 (e_{0123}): pseudoscalars in the G.A. \\

The multivector representation associated to the Geometric Algebra G_{3,0,1} will thus be composed of a total of 16 elements.

Scalars will be represented by grade 0 elements, planes by grade 1 elements, lines by grade 2 elements, points by grade 3 elements and pseudoscalar by grade 4 elements.

In [ ]:
def embed_pos_mv_16(pos):
    """
    Embeds the pos torch tensor as a 16-dimensional G_{3,0,1} multivector

    Args:
        pos (torch.Tensor): tensor of input points with shape (n_elements, 3)

    Returns:
        mv torch.Tensor: tensor of multivectors of dimension 16 with shape (1, n_elements, 1, 16)
    """
    # Positions in Euclidean geometry can be embedded as points in G_{3,0,1} algebra

    # Get the number of points and the dimensionality
    n_elements = pos.shape[0]
    dim = pos.shape[1]

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 14] = 1 # homogeneous component, e_{123}
    multivector[:, 11] = pos[:, 0] # x, e_{023}
    multivector[:, 12] = pos[:, 1] # y, e_{013}
    multivector[:, 13] = pos[:, 2] # z, e_{012}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv


# ???: question about sign convention
# the choice of the sign allow to give the correct geometric interpretation
#       to the components. Important to have the convention be consistent
#       for the whole embedding

In [ ]:
def embed_face_mv_16(face):
    """
    Embeds the face torch tensor as a 16-dimensional G_{3,0,1} multivector

    Input:
        face torch.Tensor of size (n_elements, 3)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 16)
    """

    # An oriented surface in Euclidean geometry characterized by
    #   the normal vector n to the surface itself can be embedded as an oriented
    #   plane in G_{3,0,1} algebra


    # Get the number of points and the dimensionality of the space
    n_elements = face.shape[0]
    dim = face.shape[1]

    # Create multivector tensor
    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 2] = face[:, 0] # e_{1}
    multivector[:, 3] = face[:, 1] # e_{2}
    multivector[:, 4] = face[:, 2] # e_{3}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [ ]:
def embed_wss_mv_16(wss):
    """
    Embeds the wss vectors as a 16 dimensional G_{3,0,1} multivector
    Input:
      wss torch.Tensor of size (n_elements, 3)
    Output:
      mv torch.Tensor of size (1, n_elements, 1, 16)
    """

    # A 3D vector in the Euclidean space can be embedded as a translation in G_{3,0,1} algebra

    n_elements = wss.shape[0]
    dim = wss.shape[1]

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])

    multivector[:, 0] = 1
    multivector[:, 5] = 0.5*wss[:, 0] # e_{01}
    multivector[:, 6] = 0.5*wss[:, 1] # e_{02}
    multivector[:, 7] = 0.5*wss[:, 2] # e_{03}

    mv = multivector.reshape(1,n_elements,1,global_variables['dim_GA'])

    return mv

In [ ]:
def embed_inlet_mv_16(inlet):
    """
    Embeds the inlet torch tensor as a 16 dimensional G_{3,0,1} multivector
    Input:
        inlet torch.Tensor of size (n_elements, 1)
    Output:
        mv torch.Tensor of size (1, n_elements, 1, 16)
    """
    # Inlet indexes can be embedded as scalars in G_{3,0,1} algebra

    n_elements = inlet.shape[0]

    multivector = torch.zeros(n_elements, global_variables['dim_GA'])
    multivector[:, 0] = inlet[:]

    mv = multivector.reshape(1, n_elements, 1, global_variables['dim_GA'])

    return mv

In [ ]:
def embed_pressure_mv_16(pressure):
  """
  Embeds the pressure torch tensor as a 16-dimensional G_{3,0,1} multivector
  Input:
    pressure torch.Tensor of size (n_elements, 1)
  Output:
    mv torch.Tensor of size (1, n_elements, 1, 16)
  """

  n_elements = pressure.shape[0]

  multivector = torch.zeros(n_elements, global_variables['dim_GA'])
  multivector[:,0] = pressure[:]

  mv = multivector.reshape(1, n_elements, 1, global_variables['dim_GA'])

  return mv

In [ ]:
def embed_complete_mv_16(sample_path):
  # passa alla funzione il sample path,
  # fagli estrarre i tensori in torch,
  # fai l'embedding per ogni componente
  # e infine concatena tutti gli embedding

  # ritorna il multivector completo per il sample con path sample_path

  with h5py.File(sample_path, 'r') as f:
    # extract torch tensor for pos
    pos_data = f['pos'][:50]
    pos_torch = torch.tensor(pos_data[:])

    # extract torch tensor for face
    face_data = f['face'][:50]
    face_torch = torch.tensor(face_data[:])

    # extract torch tensor for wss
    wss_data = f['wss'][:50]
    wss_torch = torch.tensor(wss_data[:])

    # extract torch tensor for pressure
    pressure_data = f['pressure'][:50]
    pressure_torch = torch.tensor(pressure_data[:])

    # extract torch tensor for inlet_idcs
    inlet_data = f['inlet_idcs'][:50]
    inlet_torch = torch.tensor(inlet_data[:])

  # Get the multivector embeddings for each torch tensor
  pos_mv_16 = embed_pos_mv_16(pos_torch)
  face_mv_16 = embed_face_mv_16(face_torch)
  wss_mv_16 = embed_wss_mv_16(wss_torch)
  inlet_mv_16 = embed_inlet_mv_16(inlet_torch)
  pressure_mv_16 = embed_pressure_mv_16(pressure_torch)

  sample_embedding = torch.cat(
      [pos_mv_16, face_mv_16, wss_mv_16, inlet_mv_16, pressure_mv_16], dim = 2)

  return sample_embedding


In [ ]:
# Use of embed_complete_mv_16
sample_path = "/content/sample_0000.hdf5"
sample_embedding = embed_complete_mv_16(sample_path)
print(sample_embedding.shape)

torch.Size([1, 50, 5, 16])


# Geometric Algebra Transformations

In [21]:
# plane reflection given normal to the plane and origin shift

def embed_plane_reflection_mv_16(normal, d):
  """
  Embeds a plane reflection transformation given a normal vector and an origin shift
  as a 16-dimensional multivector in G_{3,0,1}
  Inputs:
    normal: torch.Tensor (1, 3) normal to the plane
    d: origin shift
  Output:
    pr_mv_16: torch.Tensor multivector of size (1,1,1,16) representing the plane reflection
  """

  plane_reflection_mv_16 = torch.zeros(1, global_variables['dim_GA'])
  plane_reflection_mv_16[0,1] = d
  plane_reflection_mv_16[0,2] = normal[0]
  plane_reflection_mv_16[0,3] = normal[1]
  plane_reflection_mv_16[0,4] = normal[2]

  pr_mv_16 = plane_reflection_mv_16.reshape(1,1,1,global_variables['dim_GA'])

  return pr_mv_16



In [26]:
# translation by t in R^{3}
def embed_translation_mv_16(t):
  """
  Embeds a translation by a vector t in R^{3} as a 16-dimensional multivector in G_{3,0,1}
  Inputs:
    t: torch.Tensor of size (1, 3) representing a translation in the 3D space
  Outputs:
    tr_mv_16: torch.Tensor multivector of size (1,1,1,16) representing the translation
  """
  translation_mv_16 = torch.zeros(1, global_variables['dim_GA'])

  translation_mv_16[0,0] = 1 # homogeneous component
  translation_mv_16[0,5] = 0.5*t[0]
  translation_mv_16[0,6] = 0.5*t[1]
  translation_mv_16[0,7] = 0.5*t[2]

  tr_mv_16 = translation_mv_16.reshape(1,1,1,global_variables['dim_GA'])

  return tr_mv_16



In [27]:
t = torch.tensor([2, 4, 6])
d = 4
translation_mv = embed_translation_mv_16(t)

print(translation_mv)

tensor([[[[1., 0., 0., 0., 0., 1., 2., 3., 0., 0., 0., 0., 0., 0., 0., 0.]]]])


In [1]:
#### prova blade

mv_dimension = 16
grade_components = [1, 4, 6, 4, 1]
blade_shape = (mv_dimension,mv_dimension) # (16,16)

coordinates = []
start = 0
for length in grade_components:
    coordinates.append(list(range(start, start + length)))
    start += length
print(coordinates)


[[0], [1, 2, 3, 4], [5, 6, 7, 8, 9, 10], [11, 12, 13, 14], [15]]


In [2]:
w_dimension = len(grade_components)
print(w_dimension) # 5 (number of blades)


5


In [5]:
import torch
blade_mask = []

w_dimension = len(grade_components) # 5
for k_grade in range(w_dimension):
    print('k grade')
    print(k_grade)
    w_blade = torch.zeros(blade_shape) # per ogni grado di k genero una w_blade
    for coordinate in coordinates[k_grade]:
        w_blade[coordinate, coordinate] = 1.0
    blade_mask.append(w_blade.unsqueeze(0))
print(blade_mask) # sono 5 matrici 16 x 16 (una per ogni blade), ciascuna ha elementi =1 sulla diagonale maggiore in corrispondenza degli indici della blade!!!

k grade
0
k grade
1
k grade
2
k grade
3
k grade
4
[tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [7]:
coord_permutations = [
        [[0,1]],
        [[2,5],[3,6],[4,7]],
        [[8,11],[9,12],[10,13]],
        [[14,15]]
    ]

v_dimension = len(grade_components) - 1
for k_grade in range(v_dimension):
    v_blade = torch.zeros(blade_shape) # 16 x 16
    for coord_to,coord_from in coord_permutations[k_grade]: # 1,2,3,4
        print('coord_from: ', coord_from)
        print('coord_to: ',coord_to)
        v_blade[coord_from, coord_to] = 1.0
    print('end internal for')
    blade_mask.append(v_blade.unsqueeze(0))

blade_operator = torch.cat(blade_mask,dim = 0)

coord_from:  1
coord_to:  0
end internal for
coord_from:  5
coord_to:  2
coord_from:  6
coord_to:  3
coord_from:  7
coord_to:  4
end internal for
coord_from:  11
coord_to:  8
coord_from:  12
coord_to:  9
coord_from:  13
coord_to:  10
end internal for
coord_from:  15
coord_to:  14
end internal for
